# Layer parameters Quiz!

Idea and functionality taken from the [einops](https://github.com/arogozhnikov/einops/blob/main/docs/2-einops-for-deep-learning.ipynb) tutorials.

In [ ]:
from IPython.display import display_html

_style_inline = """<style>
.conv-answer {
    color: transparent;
    padding: 5px 15px;
    background-color: lightgray;
}
div.overfitting-container {
    width: 650px;
    height: 450px;
    background-color: white;
    position: relative;
    overflow: hidden;
}
img.overfitting, img.overfitting-answer {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    transition: opacity 0.5s ease-in-out, visibility 0.5s ease-in-out;
}
img.overfitting {
    opacity: 1;
    visibility: visible;
}
img.overfitting-answer {
    opacity: 0;
    visibility: hidden;
}
div.overfitting-container:hover img.overfitting {
    opacity: 0;
    visibility: hidden;
}
div.overfitting-container:hover img.overfitting-answer {
    opacity: 1;
    visibility: visible;
}
</style>
"""

def guess(x):
    q, a = x  # 'q' is the default image, 'a' is the image with the answer
    # Wrap both images inside the div container
    answ = f"""
    <div class='overfitting-container'>
        <img src='{q}' class='overfitting'>
        <img src='{a}' class='overfitting-answer'>
    </div>
    """
    return display_html(_style_inline + answ, raw=True)

In [ ]:
import random
import numpy as np
import tensorflow as tf

import io
import base64
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# load dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [ ]:
# Function to save the figure as an image and return the image in base64 format
def plot_epochs(y_train, y_validation, title, y_label, answer=False, display=False):
    x = range(1, len(y_train) + 1)
    fig, ax = plt.subplots()
    ax.plot(x, y_train, label="training")
    ax.plot(x, y_validation, label="validation")
    
    if answer:
        min_epoch = np.argmin(y_train)
        min_val_epoch = np.argmin(y_validation)
        # if validation is lower than train: underfitting!
        if y_validation[min_val_epoch] <= y_train[min_epoch]:
           ax.text(
               min_val_epoch + .8, y_validation[min_val_epoch] + .05,
               f"underfitting!"
           )        
        # overfitting
        else:
           ax.text(
               min_val_epoch + .8, y_validation[min_val_epoch] + .05,
                f"overfitting!"
           )
        ax.text(
           min_val_epoch + .8, y_validation[min_val_epoch] + .035,
           f"min val loss:"
        )                  
        ax.text(
            min_val_epoch + .8,
            y_validation[min_val_epoch] + .02,
            f"↓ epoch {min_val_epoch + 1}"
        )
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    ax.set_xticks(range(1, len(y_train) + 1))    
    ax.set_xlabel("Epochs")
    ax.set_xlim(left=0)
    ax.legend()

    # no display: return as a base64 string
    if not display:
        # Save the figure to a bytes buffer
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        plt.close(fig)
    
        # Convert the image to a base64 string to embed in HTML
        img_base64 = base64.b64encode(buf.read()).decode('utf-8')
        img_str = f"data:image/png;base64,{img_base64}"
        return img_str
    else:
        plt.show()

In [ ]:
def random_model():
    n_layers = random.randint(1,4)    
    hidden_units = []
    r = random.randint(2,8)
    hidden_units.append(2**r)
    for _ in range(n_layers-1):
        r = random.randint(r,8)
        hidden_units.append(2**r)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input((28*28,)))
    for i in range(n_layers):
        model.add(tf.keras.layers.Dense(hidden_units[i], activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy", 
        metrics=["accuracy"]
    )
    
    layers_str = "\n".join(
        [f'model.add(tf.keras.layers.Dense({hidden_units[i]}, activation="relu"))' for i in range(n_layers)]
    )
    
    print(f"""
reset_random_seeds()
model = tf.keras.models.Sequential()
model.add(tf.keras.Input((28*28,)))
{layers_str}
model.add(tf.keras.layers.Dense(10, activation="softmax"))
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)
history = model.fit(
    train_images, train_labels, epochs=20,
    validation_split=0.2, verbose=0
)
plot_epochs(
    history.history["loss"], history.history["val_loss"],
    title='Training and validation loss',
    y_label='Loss', answer=True, display=True
)
    """)

    print("Training...")
    
    # model.summary()
    history = model.fit(
        train_images, train_labels, epochs=20,
        validation_split=0.2, verbose=0
    )
    figure_base64 = plot_epochs(
        history.history["loss"], history.history["val_loss"],
        title='Training and validation loss',
        y_label='Loss'
    )
    figure_answer_base64 = plot_epochs(
        history.history["loss"], history.history["val_loss"],
        title='Training and validation loss',
        y_label='Loss', answer=True
    )

    print()
    print("Consider the following plot: at which epoch does overfitting happen? (Hover to reveal.)")
    print()
    print("(Copy the above code and run it if you want to train another model with the same hyperparameters.)")
    print()

    return figure_base64,figure_answer_base64

## The Quiz

Rerun for a new set-up, hover to see the answer.

In [ ]:
guess(random_model())